## Algoritmo para otimização

In [21]:
import random
import math

# Parâmetros do problema
BOUNDS = {'h': (10.0, 100.0), 't': (1.0, 10.0), 's': (2.0, 20.0)}
PARAM_KEYS = ['h', 't', 's']

# Constantes do Algoritmo
TAM_POP = 20  #TAMANHO DA POPULAÇÃO
N_GER = 100 #N_GERAÇÕES
P_CROSSOVER = 0.8 #PROBABILIDADE DE CRUZAMENTO
P_MUT = 0.1 # PROBABILIDADE DE MUTAÇÃO
K_TORNEIO = 3 # Tamanho do torneio
TAMANHO_ELITE = 1 # Tamanho da elite

# Pesos das penalizações (mais fáceis de ajustar aqui)
PENALTY_WEIGHT_VOL = 10.0
PENALTY_WEIGHT_SUM = 10.0

# ===== Funções básicas =====

def criar_individuo():
    return [random.uniform(*BOUNDS[k]) for k in PARAM_KEYS]

def limitar_genes(ind):
    return [max(min(g, BOUNDS[k][1]), BOUNDS[k][0]) for g, k in zip(ind, PARAM_KEYS)]

def func_desempenho(h, t, s):
    return (h * math.log(s + 1)) / (t + 1)

def avaliar_individuo(ind):
    h, t, s = ind
    desempenho = func_desempenho(h, t, s)
    
    # Penalizações por violação das restrições
    viol_vol = max(0.0, h * t * s - 10000.0)
    viol_sum = max(0.0, (t + s) - 25.0)
    
    penalty = PENALTY_WEIGHT_VOL * viol_vol + PENALTY_WEIGHT_SUM * viol_sum
    return desempenho - penalty

# ===== População =====

def criar_populacao(n_individuos):
    return [criar_individuo() for _ in range(n_individuos)]

# ===== Operadores genéticos =====

def selecao_torneio(pop, fits, k):
    idxs = random.sample(range(len(pop)), k)
    best_idx_in_sample = max(idxs, key=lambda i: fits[i])
    return pop[best_idx_in_sample]

def crossover(pai, mae, fator_blend=0.5): # Variável 'alpha' renomeada para clareza
    filho = [fator_blend * x + (1 - fator_blend) * y for x, y in zip(pai, mae)]
    return limitar_genes(filho)

def mutacao_gauss(ind, sigma=0.1, p_mut_gene=0.1):
    novo_ind = []
    for gene, key in zip(ind, PARAM_KEYS):
        if random.random() < p_mut_gene:
            low, high = BOUNDS[key]
            gene += random.gauss(0, sigma * (high - low))
        novo_ind.append(gene)
    return limitar_genes(novo_ind)

# ===== Geração seguinte (Lógica de avaliação da elite corrigida) =====

def criar_nova_geracao(pop, fits):
    N = len(pop)
    
    # Elitismo: Mantém os melhores indivíduos E SEUS SCORES DE FITNESS
    idx_sorted = sorted(range(N), key=lambda i: fits[i], reverse=True)
    
    # Copia a elite (indivíduos e fitness) para a nova geração
    nova_pop = [pop[i] for i in idx_sorted[:TAMANHO_ELITE]]
    novos_fits = [fits[i] for i in idx_sorted[:TAMANHO_ELITE]]

    # Preenche o restante da população
    while len(nova_pop) < N:
        pai = selecao_torneio(pop, fits, k=K_TORNEIO)
        mae = selecao_torneio(pop, fits, k=K_TORNEIO)
        
        if random.random() < P_CROSSOVER:
            filho = crossover(pai, mae)
        else:
            filho = pai[:] # Clona o pai
            
        filho = mutacao_gauss(filho, p_mut_gene=P_MUT)
        
        nova_pop.append(filho)
        # IMPORTANTE: Avalia apenas o novo filho gerado
        novos_fits.append(avaliar_individuo(filho))
        
    return nova_pop, novos_fits

# ===== Execução =====
if __name__ == '__main__':
    # População inicial
    populacao = criar_populacao(TAM_POP)
    fitnesses = [avaliar_individuo(ind) for ind in populacao]

    print(populacao)
    print(fitnesses)
    print("Iniciando otimização...")
    # Loop de gerações
    for gen in range(N_GER):
        populacao, fitnesses = criar_nova_geracao(populacao, fitnesses)
        
        # Análise da geração
        i_best = max(range(TAM_POP), key=lambda i: fitnesses[i])
        ind_best = populacao[i_best]
        
        h,t,s = ind_best
        desempenho_puro = func_desempenho(h,t,s)
        
        print(f"G{gen:03d}: Melhor Fitness={fitnesses[i_best]:.4f} | "
              f"Desempenho Puro={desempenho_puro:.4f} | "
              f"h*t*s={h*t*s:.2f} | t+s={t+s:.2f} | "
              f"h={h:.2f}, t={t:.2f}, s={s:.2f}")

    # Resultado final
    i_final_best = max(range(TAM_POP), key=lambda i: fitnesses[i])
    melhor_final = populacao[i_final_best]
    h, t, s = melhor_final
    
    print("\n--- Otimização Concluída ---")
    print(f"Melhor Fitness Encontrado: {fitnesses[i_final_best]:.4f}")
    print(f"Melhores Parâmetros (h, t, s): ({h:.4f}, {t:.4f}, {s:.4f})")
    print(f"Restrição de Volume (h*t*s < 10000): {h*t*s:.2f}")
    print(f"Restrição de Espaço (t+s < 25): {t+s:.2f}")


[[41.12376278402538, 2.548608868121154, 16.836904931964106], [45.1655051188266, 9.20910974033799, 14.31538938623577], [31.107113416709552, 7.436156448716582, 16.487214026566594], [34.87115473880962, 2.795107101919243, 6.014202833316904], [18.401581420864126, 1.612358121321546, 9.83988706166426], [32.333973992816325, 8.132968467754594, 13.092951918597791], [87.96453237309795, 5.276900739518304, 4.7624747928085], [17.079376532550103, 3.6459924437705293, 12.385381066320335], [35.01042276044273, 6.482751876306592, 8.972725382756822], [63.733053821764095, 2.8835115439152377, 8.358038425053655], [25.72894375726184, 1.0320837871230135, 3.3761115994994935], [52.03600345201641, 8.682578184946214, 19.37544893180789], [34.20541314407069, 1.516185348455367, 9.261141013143135], [37.647266282312, 1.44841598199678, 16.554195946971383], [58.92894046126082, 1.5633146087554834, 18.114835552581646], [36.877241310379674, 4.246254609912553, 16.35772305335192], [38.43074618967795, 3.5156471467944708, 19.764

## Algoritmo para otimização dos hiperparâmetros

#### A otimização não mostrou muita mudança observável na velocidade de convergência, tendo side testado com os parametrosd otimizados e não otimizados

In [8]:
import optuna

def objective(trial):
    # Hiperparâmetros a otimizar
    tam_pop = trial.suggest_int("TAM_POP", 10, 100)
    n_ger = trial.suggest_int("N_GER", 50, 200)
    p_cross = trial.suggest_float("P_CROSSOVER", 0.5, 1.0)
    p_mut = trial.suggest_float("P_MUT", 0.01, 0.5)
    k_torneio = trial.suggest_int("K_TORNEIO", 2, 5)
    
    # Substitui os parâmetros globais
    global TAM_POP, N_GER, P_CROSSOVER, P_MUT, K_TORNEIO
    TAM_POP = tam_pop
    N_GER = n_ger
    P_CROSSOVER = p_cross
    P_MUT = p_mut
    K_TORNEIO = k_torneio
    
    # Rodar o algoritmo com esses hiperparâmetros
    populacao = criar_populacao(TAM_POP)
    fitnesses = [avaliar_individuo(ind) for ind in populacao]

    for _ in range(N_GER):
        populacao, fitnesses = criar_nova_geracao(populacao, fitnesses)

    best_fit = max(fitnesses)
    return -best_fit  # Minimização (Optuna sempre minimiza)


In [9]:
if __name__ == "__main__":
    study = optuna.create_study()
    study.optimize(objective, n_trials=50)

    print("Melhores parâmetros encontrados:")
    print(study.best_params)


[I 2025-07-10 16:42:57,743] A new study created in memory with name: no-name-41a4c200-7b6f-4c5c-bb43-8b7c617e7959
[I 2025-07-10 16:42:57,785] Trial 0 finished with value: -152.22612188617114 and parameters: {'TAM_POP': 73, 'N_GER': 58, 'P_CROSSOVER': 0.8001604424678442, 'P_MUT': 0.2586683284288873, 'K_TORNEIO': 5}. Best is trial 0 with value: -152.22612188617114.
[I 2025-07-10 16:42:57,858] Trial 1 finished with value: -152.22612188617114 and parameters: {'TAM_POP': 78, 'N_GER': 112, 'P_CROSSOVER': 0.7848348056456069, 'P_MUT': 0.052307743005936616, 'K_TORNEIO': 5}. Best is trial 0 with value: -152.22612188617114.
[I 2025-07-10 16:42:57,892] Trial 2 finished with value: -152.22612188617114 and parameters: {'TAM_POP': 37, 'N_GER': 116, 'P_CROSSOVER': 0.8709659869933928, 'P_MUT': 0.08947858410960027, 'K_TORNEIO': 2}. Best is trial 0 with value: -152.22612188617114.
[I 2025-07-10 16:42:57,964] Trial 3 finished with value: -152.22612188617114 and parameters: {'TAM_POP': 65, 'N_GER': 127, 'P

Melhores parâmetros encontrados:
{'TAM_POP': 73, 'N_GER': 58, 'P_CROSSOVER': 0.8001604424678442, 'P_MUT': 0.2586683284288873, 'K_TORNEIO': 5}
